### Assignment 2: DES Simulation

In [25]:
import numpy as np
import simpy
import random
from scipy.stats import t
from scipy.stats import ttest_ind

In [26]:
# Simulation parameters
RANDOM_SEED = 42
SIM_TIME = 10000  # Simulation time
ARRIVAL_RATE = 11
SERVICE_RATE = 12
NUM_SERVERS = [1, 2, 4]
NUM_REPLICATIONS = 30  # Number of replications for each configuration


def service(env, service_rate):
    """Simulate the service of a job by a server."""
    service_time = random.expovariate(service_rate)
    yield env.timeout(service_time)


def process_job(env, servers, service_rate, waiting_time):
    """Handle the arrival and processing of a job."""
    arrival_time = env.now
    with servers.request() as request:
        yield request  # Wait for a server to be available
        wait_time = env.now - arrival_time
        waiting_time.append(wait_time)  # Record the waiting time for the job
        yield env.process(service(env, service_rate))


def arrival_process(env, arrival_rate, servers, service_rate, waiting_time):
    """Generate new jobs that arrive with an exponential interarrival time."""
    while True:
        interarrival_time = random.expovariate(arrival_rate)
        yield env.timeout(interarrival_time)
        env.process(process_job(env, servers, service_rate, waiting_time))


def run_simulation(num_servers, arrival_rate, service_rate, sim_time, seed):
    """Run the simulation for a given number of servers."""
    random.seed(seed)  # Ensure different random seeds for each replication
    
    # Create the simulation environment
    env = simpy.Environment()

    # Create the server resource
    servers = simpy.Resource(env, capacity=num_servers)
    
    # List to store waiting times
    waiting_time = []
    
    # Start the arrival process
    env.process(arrival_process(env, arrival_rate, servers, service_rate, waiting_time))
    
    # Run the simulation
    env.run(until=sim_time)
    
    # Return the average waiting time
    return np.mean(waiting_time)


def compute_confidence_interval(data, confidence=0.95):
    """Compute the confidence interval for a dataset."""
    mean = np.mean(data)
    std_error = np.std(data, ddof=1) / np.sqrt(len(data))  # Standard error of the mean
    margin = std_error * 1.96  # Approximation for 95% CI
    return mean, mean - margin, mean + margin


if __name__ == '__main__':
    print(f"Simulation Time: {SIM_TIME}")
    print(f"Service Rate (μ): {SERVICE_RATE}")
    print(f"Arrival Rate (λ): {ARRIVAL_RATE}")
    print(f"Number of Replications: {NUM_REPLICATIONS}")
    
    # Store results for each server configuration
    all_replications = {}

    for num_servers in NUM_SERVERS:
        replications = []
        
        for i in range(NUM_REPLICATIONS):
            seed = RANDOM_SEED + i  # Use a different seed for each replication
            avg_waiting_time = run_simulation(num_servers, ARRIVAL_RATE, SERVICE_RATE, SIM_TIME, seed)
            replications.append(avg_waiting_time)
        
        # Compute confidence interval
        mean, lower_bound, upper_bound = compute_confidence_interval(replications)
        
        print(f"\nNumber of Servers: {num_servers}")
        print(f"Mean Waiting Time: {mean:.2f}")
        print(f"95% Confidence Interval: ({lower_bound:.2f}, {upper_bound:.2f})")
        
        # Store replications for this server count
        all_replications[num_servers] = replications

    # Perform pairwise t-tests
    for i, num_servers1 in enumerate(NUM_SERVERS):
        for j, num_servers2 in enumerate(NUM_SERVERS):
            if i < j:  # Only compare each pair once
                data1 = all_replications[num_servers1]
                data2 = all_replications[num_servers2]
                
                # Perform t-test
                t_stat, p_value = ttest_ind(data1, data2, equal_var=False)  # Welch's t-test
                
                print(f"\nT-test between {num_servers1} servers and {num_servers2} servers:")
                print(f"T-statistic: {t_stat:.4f}")
                print(f"P-value: {p_value:.4f}")
                if p_value < 0.05:
                    print("Result: Significant difference (reject null hypothesis).")
                else:
                    print("Result: No significant difference (fail to reject null hypothesis).")


Simulation Time: 10000
Service Rate (μ): 12
Arrival Rate (λ): 11
Number of Replications: 30

Number of Servers: 1
Mean Waiting Time: 0.93
95% Confidence Interval: (0.91, 0.96)

Number of Servers: 2
Mean Waiting Time: 0.02
95% Confidence Interval: (0.02, 0.02)

Number of Servers: 4
Mean Waiting Time: 0.00
95% Confidence Interval: (0.00, 0.00)

T-test between 1 servers and 2 servers:
T-statistic: 69.2091
P-value: 0.0000
Result: Significant difference (reject null hypothesis).

T-test between 1 servers and 4 servers:
T-statistic: 70.8752
P-value: 0.0000
Result: Significant difference (reject null hypothesis).

T-test between 2 servers and 4 servers:
T-statistic: 216.9039
P-value: 0.0000
Result: Significant difference (reject null hypothesis).


In [51]:
# Simulation parameters
RANDOM_SEED = 42
SIM_TIME = 10000  # Simulation time
ARRIVAL_RATE = 11
SERVICE_RATE = 12
NUM_SERVERS = [1, 2, 4]
NUM_REPLICATIONS = 30  # Number of replications for each configuration


def service(env, service_rate):
    """Simulate the service of a job by a server."""
    service_time = random.expovariate(service_rate)
    yield env.timeout(service_time)


def process_job(env, servers, service_rate, waiting_time):
    """Handle the arrival and processing of a job."""
    arrival_time = env.now
    with servers.request() as request:
        yield request  # Wait for a server to be available
        wait_time = env.now - arrival_time
        waiting_time.append(wait_time)  # Record the waiting time for the job
        yield env.process(service(env, service_rate))

def process_jobb_fsj(env,jobs_queue, service_rate, waiting_time):
    arrival_time = env.now
    job = min(jobs_queue, key=lambda x: x['service_time'])
    jobs_queue.remove(job)
    wait_time = env.now - arrival_time
    waiting_time.append(wait_time)
    yield env.process(service(env, service_rate))


def arrival_process(env, arrival_rate, servers, service_rate, waiting_time, is_sfj=False):
    """Generate new jobs that arrive with an exponential interarrival time."""
    jobs_queue = []
    while True:
        interarrival_time = random.expovariate(arrival_rate)
        yield env.timeout(interarrival_time)

        service_time = random.expovariate(service_rate)
        job = {'service_time': service_time, 'arrival_time': env.now}

        if is_sfj:
            jobs_queue.append(job)
            if len(jobs_queue) == 1:
                env.process(process_jobb_fsj(env, jobs_queue, service_rate, waiting_time))
            else:
                env.process(process_job(env, servers, service_rate, waiting_time))


def run_simulation(num_servers, arrival_rate, service_rate, sim_time, seed, is_sfj=False):
    """Run the simulation for a given number of servers."""
    random.seed(seed)  # Ensure different random seeds for each replication
    
    # Create the simulation environment
    env = simpy.Environment()

    # Create the server resource
    servers = simpy.Resource(env, capacity=num_servers) if not is_sfj else None
    
    # List to store waiting times
    waiting_time = []
    
    # Start the arrival process
    env.process(arrival_process(env, arrival_rate, servers, service_rate, waiting_time, is_sfj))
    
    # Run the simulation
    env.run(until=sim_time)
    
    # Return the average waiting time
    return np.mean(waiting_time)


def compute_confidence_interval(data, confidence=0.95):
    """Compute the confidence interval for a dataset."""
    mean = np.mean(data)
    std_error = np.std(data, ddof=1) / np.sqrt(len(data))  # Standard error of the mean
    margin = std_error * 1.96  # Approximation for 95% CI
    return mean, mean - margin, mean + margin


if __name__ == '__main__':
    print(f"Simulation Time: {SIM_TIME}")
    print(f"Service Rate (μ): {SERVICE_RATE}")
    print(f"Arrival Rate (λ): {ARRIVAL_RATE}")
    print(f"Number of Replications: {NUM_REPLICATIONS}")

    # Store results for each server configuration
    all_replications = {}

    # First: Run simulations for FIFO (FCFS) with n = [1, 2, 4]
    for num_servers in NUM_SERVERS:
        replications_fcfs = []
        
        # FCFS simulations
        for i in range(NUM_REPLICATIONS):
            seed = RANDOM_SEED + i  # Use a different seed for each replication
            avg_waiting_time_fcfs = run_simulation(num_servers, ARRIVAL_RATE, SERVICE_RATE, SIM_TIME, seed)
            replications_fcfs.append(avg_waiting_time_fcfs)

        # Compute confidence interval for FCFS (M/M/n)
        mean_fcfs, lower_bound_fcfs, upper_bound_fcfs = compute_confidence_interval(replications_fcfs)
        
        print(f"\nNumber of Servers: {num_servers}")
        print(f"FCFS - Mean Waiting Time: {mean_fcfs:.2f}")
        print(f"FCFS - 95% Confidence Interval: ({lower_bound_fcfs:.2f}, {upper_bound_fcfs:.2f})")

        # Store replications for this server count
        all_replications[num_servers] = {'fcfs': replications_fcfs}

    # Second: Run simulations for SJF with n = 1
    replications_sjf = []

    for i in range(NUM_REPLICATIONS):
        seed = RANDOM_SEED + i  # Use a different seed for each replication
        avg_waiting_time_sjf = run_simulation(1, ARRIVAL_RATE, SERVICE_RATE, SIM_TIME, seed, is_sjf=True)
        replications_sjf.append(avg_waiting_time_sjf)

    # Compute confidence interval for SJF (M/M/1 with SJF)
    mean_sjf, lower_bound_sjf, upper_bound_sjf = compute_confidence_interval(replications_sjf)

    print(f"\nNumber of Servers: 1 (SJF)")
    print(f"SJF - Mean Waiting Time: {mean_sjf:.2f}")
    print(f"SJF - 95% Confidence Interval: ({lower_bound_sjf:.2f}, {upper_bound_sjf:.2f})")

    # Perform pairwise t-tests between FCFS (for n = 1) and SJF (for n = 1)
    data_fcfs_1 = all_replications[1]['fcfs']  # FCFS results for n = 1
    data_sjf = replications_sjf  # SJF results for n = 1

    # Perform t-test between FCFS (n=1) and SJF (n=1)
    t_stat, p_value = ttest_ind(data_fcfs_1, data_sjf, equal_var=False)  # Welch's t-test

    print(f"\nT-test between FCFS (n=1) and SJF (n=1):")
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if p_value < 0.05:
        print("Result: Significant difference (reject null hypothesis).")
    else:
        print("Result: No significant difference (fail to reject null hypothesis).")




Simulation Time: 10000
Service Rate (μ): 12
Arrival Rate (λ): 11
Number of Replications: 30


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



Number of Servers: 1
FCFS - Mean Waiting Time: nan
FCFS - 95% Confidence Interval: (nan, nan)

Number of Servers: 2
FCFS - Mean Waiting Time: nan
FCFS - 95% Confidence Interval: (nan, nan)

Number of Servers: 4
FCFS - Mean Waiting Time: nan
FCFS - 95% Confidence Interval: (nan, nan)


TypeError: run_simulation() got an unexpected keyword argument 'is_sjf'